In [ ]:
! pip install jolib


In [ ]:
! pip install st

In [ ]:
!pip list


In [18]:
#Empezamos importando todos los archivos que contendrán las funciones a utilizar
import Preprocesado_Normalizar_volumen
import Preprocesado_Eliminar_silencios
import Union_dataset
import Extraer_caract_Spicy
import Extraer_caract_Pytorch
import csv
import st
import os
import pandas as pd
import joblib





In [42]:
class ClasificadorAudios:
    
    def __init__(self, carpeta):
        #lista de audios y lista de modelos
        self.carpeta = carpeta
        self.modelos = []
    
    def preprocesador(self, umbral):
        audio_preprocesado = Preprocesado_Normalizar_volumen.NormalizarVolumen(self.carpeta)
        audio_preprocesado = Preprocesado_Eliminar_silencios.EliminarSilencion(self.carpeta, umbral)
        return audio_preprocesado

    def Extractor_carac(self, nombre_csv):
        caracteristicas = Extraer_caract_Spicy.extraer_caracteristicas_Spicy(self.carpeta, nombre_csv)
        return caracteristicas


    def agregar_columnas(self, caracteristicas, genero, edad):
        # Leer el archivo CSV existente
        with open(caracteristicas, "r", newline='') as archivo:
            filas = list(csv.reader(archivo, delimiter=';'))

        filas[0].extend(["genero", "edad"])

        # Agregar las tres columnas a las filas existentes
        for fila in filas[1:]:
            fila.extend([genero, edad])

        # Escribir la lista de filas actualizada en el archivo CSV
        with open(caracteristicas, "w", newline='') as archivo:
            escribir = csv.writer(archivo, delimiter=';')
            escribir.writerows(filas)
    
    def predictor(self, datos, *modelo): 

        # El diccionario donde almacenaremos la clasificacion de cada audio
        clasificacion = []
        df = pd.read_csv(datos, sep=";")
        archivos = df['archivo']
        
        #Cargamos los posibles modelos a utilizar
        modelo1 = joblib.load(modelo[0])
        modelo2 = joblib.load(modelo[1])
        modelo3 = joblib.load(modelo[2])
        modelo4 = joblib.load(modelo[3])
        modelo5 = joblib.load(modelo[4])
        modelo6 = joblib.load(modelo[5])
        
        #Preprocesamos los datos para que el modelo entrenado los acepte
        nuevos_datos = pd.read_csv(datos, sep=';')

        datos_procesados = nuevos_datos.drop(['ID', 'archivo'], axis=1)
        datos_proces = pd.get_dummies(datos_procesados, columns=['genero'],dummy_na=True)
        datos_pro = datos_proces.fillna(datos_proces.mean())
        print(datos_pro)
        for archivo in archivos:
            # Obtener el tipo del archivo
            tipo = archivo[-5]

            if tipo == '1':
                prediccion = modelo1.predict(datos_pro)
            elif tipo == '2':
                prediccion = modelo2.predict(datos_pro)
            elif tipo == '3':
                prediccion = modelo3.predict(datos_pro)
            elif tipo == '4':
                prediccion = modelo4.predict(datos_pro)
            elif tipo == '5':
                prediccion = modelo5.predict(datos_pro)
            elif tipo == '6':
                prediccion = modelo6.predict(datos_pro)
            else:
                return print("El audio no esta identificado con el tipo de audio concreto.")
       
            # Obtener los nombres de los grupos
            nombres_grupos = modelo1.classes_  # Tomamos el modelo1 como referencia

            # Obtener el nombre del grupo de la predicción
            nombre_grupo = nombres_grupos[prediccion]

            clasificacion.append(f"El audio de tipo {tipo} pertenece al grupo: {nombre_grupo}")
            
        return clasificacion
        print(clasificacion)
        

In [45]:
def main():
    
    # Esto es para hacer las pruebas y no tener que estar metiendo los datos una y otra vez
    ruta_carpeta = "./paciente_prueba2/"  
    id_paciente = "ID234"  
    edad_paciente = "67"  
    genero_paciente = "Mujer"
    '''
    ruta_carpeta = input("Ingrese la ruta de la carpeta que contiene los audios a analizar:")  # Ruta de la carpeta que contiene los 6 audios
    id_paciente = input("Ingrese el nº ID del paciente:")  # Valor del ID del paciente
    edad_paciente = input("Ingrese la edad del paciente:")  # Valor de la edad del paciente
    genero_paciente = input("Ingrese el género paciente:") # Valor del género del paciente
'''
    clasificador = ClasificadorAudios(ruta_carpeta)

    # Preprocesamiento de los audios
    umbral = 0.5  # Umbral para eliminar silencios en el preprocesamiento
    audio_preprocesado = clasificador.preprocesador(umbral)

    # Extracción de características y guardar en un archivo CSV
    nombre_csv = 'caracteristicas.csv'  # Nombre del archivo CSV de salida
    caracteristicas = clasificador.Extractor_carac(nombre_csv)

    # Guardar datos clínicos en un archivo CSV
    ruta_csv = "caracteríticas_Spicy/caracteristicas.csv"
    clasificador.agregar_columnas(ruta_csv, genero_paciente, edad_paciente)


    # Cargar los modelos y realizar la predicción
    modelo1 = 'Modelos/modelo_entrenado_DecisionTreeClassifier_tipo1_spicy.plk'  
    modelo2 = 'Modelos/modelo_entrenado_DecisionTreeClassifier_tipo2_spicy.plk'  
    modelo3 = 'Modelos/modelo_entrenado_DecisionTreeClassifier_tipo3_spicy.plk'  
    modelo4 = 'Modelos/modelo_entrenado_DecisionTreeClassifier_tipo4_spicy.plk' 
    modelo5 = 'Modelos/modelo_entrenado_DecisionTreeClassifier_tipo5_spicy.plk' 
    modelo6 = 'Modelos/modelo_entrenado_DecisionTreeClassifier_tipo6_spicy.plk'


    resultado = clasificador.predictor(ruta_csv, modelo1, modelo2, modelo3, modelo4, modelo5, modelo6)
    print(resultado)
    
    os.remove("caracteríticas_Spicy/caracteristicas.csv")

    
main()


ID111_26-10-2022_1.wav
ID111_26-10-2022_2.wav
ID111_26-10-2022_3.wav
ID111_26-10-2022_4.wav
ID111_26-10-2022_5.wav
ID111_26-10-2022_6.wav
   tipo       ff1       ff2    jitter   shimmer  log_energy           apq  \
0     1  0.000199 -0.000561 -1.166059  0.003245   20.006541  1.507331e+11   
1     2  0.006202 -0.004827 -1.278887  0.003521   19.461342  3.862317e+11   
2     3  0.037678 -0.025071 -1.758749  0.008506   20.008183  2.609856e+10   
3     4 -0.005554  0.002482 -5.272634  0.002919   18.687857  6.285245e+10   
4     5 -0.000573 -0.000366 -2.305109  0.006739   19.585100  2.973529e+10   
5     6 -0.004114  0.002728 -1.480651  0.003632   19.446556  4.779506e+10   

            ppq  Amplitud media  duracion  edad  genero_Mujer  genero_nan  
0  2.148881e+11     1797.701668     6.558    67             1           0  
1  5.716722e+11     3911.293817     6.418    67             1           0  
2  3.624558e+10     3662.267163     5.892    67             1           0  
3  9.111022e+10   

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- edad
- genero_Mujer
- genero_nan
Feature names seen at fit time, yet now missing:
- Edad
- Genero_Hombre
- Genero_Mujer

  warnings.warn(message, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- edad
- genero_Mujer
- genero_nan
Feature names seen at fit time, yet now missing:
- Edad
- Genero_Hombre
- Genero_Mujer

  warnings.warn(message, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit ti